In [8]:
# Imports
import pandas as pd
import pickle
import os
import logging
from openai import AzureOpenAI
import time
import tiktoken
from dotenv import load_dotenv
from evaluation.evaluation import AdherenceEvaluation
from utils.openai_api import get_openai_response, get_openai_response_content

SyntaxError: invalid syntax (evaluation.py, line 13)